In [6]:
import requests
import pandas as pd

def make_fr_dataframe(firm_code):
    fr_url = 'http://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701&gicode=' + firm_code
    fr_page = requests.get(fr_url)
    fr_tables = pd.read_html(fr_page.text)

    temp_df = fr_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[[
        '유동비율계산에 참여한 계정 펼치기',
        '부채비율계산에 참여한 계정 펼치기',
        '영업이익률계산에 참여한 계정 펼치기',
        'ROA계산에 참여한 계정 펼치기',
        'ROIC계산에 참여한 계정 펼치기'
    ]]
    temp_df.index = ['유동비율','부채비율','영업이익률','ROA','ROIC']
    
    return temp_df

In [7]:
def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code:dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]

        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)
            
    return total_df

In [8]:
firm_code_list = ['A005930','A035420']

for num, code in enumerate(firm_code_list):
    fr_df = make_fr_dataframe(code)
    fr_df_changed = change_df(code, fr_df)
    
    if num == 0:
        total_fr = fs_df_changed
    else:
        total_fr = pd.concat([total_fr, fr_df_changed])

In [9]:
total_fr

2015/12                          2016/12                           \
           유동비율  부채비율 영업이익률   ROA   ROIC    유동비율  부채비율 영업이익률   ROA   ROIC   
A035420   156.8  93.3  25.5  13.3  124.2   232.8  54.3  27.4  14.1  199.2   
A035420   156.8  93.3  25.5  13.3  124.2   232.8  54.3  27.4  14.1  199.2   

         ... 2018/12                        2019/12                         
         ...    유동비율  부채비율 영업이익률  ROA  ROIC    유동비율  부채비율 영업이익률  ROA  ROIC  
A035420  ...   205.6  66.1  16.9  7.0  68.6   149.6  89.1  10.8  3.6  36.6  
A035420  ...   205.6  66.1  16.9  7.0  68.6   149.6  89.1  10.8  3.6  36.6  

[2 rows x 25 columns]